In [1]:
import polars as pl

path_companies = '/Users/jannajulianfeiten/IronHack/final/companies.csv'
df_companies = pl.read_csv(path_companies)

path_unicorn = '/Users/jannajulianfeiten/IronHack/final/tech-startups-analysis/data/raw/Unicorn_Companies.csv'

try:
    df_unicorn = pl.read_csv(
        path_unicorn,
        null_values=["None"]  # Tells Polars to treat the string "None" as a missing value (null)
    )

except Exception as e:
    print(f"An error occurred trying to load the small file: {e}")

df_companies

id,Unnamed: 0.1,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,logo_url,logo_width,logo_height,short_description,description,overview,tag_list,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at,lat,lng,ROI
str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,str,str,f64,f64,str,str,str,f64,f64,f64
"""c:1""",0,"""Company""",1,null,"""Wetpaint""","""wetpaint""","""/company/wetpaint""","""web""","""operating""","""2005-10-17""",null,"""wetpaint-inc.com""","""http://wetpaint-inc.com""","""BachelrWetpaint""","""http://s3.amazonaws.com/crunch…",401.0,54.0,null,"""Technology Platform Company""","""Wetpaint is a technology platf…","""wiki, seattle, elowitz, media-…","""USA""","""WA""","""Seattle""","""Seattle""",null,null,null,null,"""2005-10-01""","""2008-05-19""",3.0,3.975e7,"""2010-09-05""","""2013-09-18""",5.0,17.0,"""initial-importer""","""2007-05-25 06:51:27""","""2013-04-13 03:29:00""",47.606209,-122.332071,15.5
"""c:10""",1,"""Company""",10,null,"""Flektor""","""flektor""","""/company/flektor""","""games_video""","""acquired""",null,null,"""flektor.com""","""http://www.flektor.com""",null,"""http://s3.amazonaws.com/crunch…",186.0,85.0,null,null,"""Flektor is a rich-media mash-u…","""flektor, photo, video""","""USA""","""CA""","""Culver City""","""Los Angeles""",null,null,null,null,null,null,null,null,null,null,null,6.0,"""initial-importer""","""2007-05-31 21:11:51""","""2008-05-23 23:23:14""",34.021122,-118.396467,null
"""c:100""",2,"""Company""",100,null,"""There""","""there""","""/company/there""","""games_video""","""acquired""",null,null,"""there.com""","""http://www.there.com""",null,"""http://s3.amazonaws.com/crunch…",107.0,34.0,null,null,"""There.com is an online virtual…","""virtualworld, there, teens""","""USA""","""CA""","""San Mateo""","""SF Bay""",null,null,null,null,null,null,null,null,"""2003-02-01""","""2011-09-23""",4.0,12.0,"""initial-importer""","""2007-08-06 23:52:45""","""2013-11-04 02:09:48""",37.562992,-122.325525,null
"""c:10000""",3,"""Company""",10000,null,"""MYWEBBO""","""mywebbo""","""/company/mywebbo""","""network_hosting""","""operating""","""2008-07-26""",null,"""mywebbo.com""","""http://www.mywebbo.com""",null,null,null,null,null,null,"""BRAND NEW ONLINE SOCIAL NETWOR…","""social-network, new, website, …",null,null,null,"""unknown""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""2008-08-24 16:51:57""","""2008-09-06 14:19:18""",null,null,null
"""c:10001""",4,"""Company""",10001,null,"""THE Movie Streamer""","""the movie streamer""","""/company/the-movie-streamer""","""games_video""","""operating""","""2008-07-26""",null,"""themoviestreamer.com""","""http://themoviestreamer.com""",null,"""http://s3.amazonaws.com/crunch…",200.0,74.0,null,null,"""This company shows free movies…","""watch, full-length, moives, on…",null,null,null,"""unknown""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""2008-08-24 17:10:34""","""2008-09-06 14:19:18""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""c:99940""",196548,"""Company""",99940,null,"""Webgility""","""webgility""","""/company/webgility""","""ecommerce""","""operating""","""2007-01-01""",null,"""webgility.com""","""http://www.webgility.com""","""webgility""","""http://s3.amazonaws.com/crunch…",200.0,54.0,null,"""eCommerce automation & integra…","""Webgility is the leading provi…","""ecommerce, quickbooks-sync, in…","""USA""","""CA""","""San Francisco""","""SF Bay""",null,null,null,null,null,null,null,null,"""2013-08-12""","""2013-08-12""",2.0,5.0,null,"""2011-09-06 2

### Cleaning df_companies

In [2]:
# Set Polars display option to show all 44 rows (we set to 50 for safety)
# This uses the correct pl.Config method for your Polars version.
pl.Config.set_tbl_rows(50)

# --- Null Percentage Calculation for df_companies ---

# 2. Get the total number of rows
total_rows = df_companies.height

# 3. Select all columns, apply the null count logic, and calculate percentage
null_counts_df = df_companies.select(
    (pl.col("*").null_count() / total_rows) * 100
)

# 4. Extract column names and percentage values
column_names = null_counts_df.columns
percentage_values = null_counts_df.row(0) 

# 5. Create the final DataFrame
null_report_final = pl.DataFrame({
    "Column Name": column_names,
    "Null Percentage (%)": pl.Series(percentage_values)
})

# 6. Sort the report to see the highest percentage of nulls first
final_report_sorted = null_report_final.sort("Null Percentage (%)", descending=True)

# Print the final report (now showing all 44 rows)
print("--- Full Null Percentage Report (df_companies) ---")
print(final_report_sorted)

--- Full Null Percentage Report (df_companies) ---
shape: (44, 2)
┌─────────────────────┬─────────────────────┐
│ Column Name         ┆ Null Percentage (%) │
│ ---                 ┆ ---                 │
│ str                 ┆ f64                 │
╞═════════════════════╪═════════════════════╡
│ parent_id           ┆ 100.0               │
│ ROI                 ┆ 99.630634           │
│ first_investment_at ┆ 98.685851           │
│ last_investment_at  ┆ 98.685851           │
│ investment_rounds   ┆ 98.68178            │
│ invested_companies  ┆ 98.68178            │
│ closed_at           ┆ 98.667026           │
│ short_description   ┆ 96.371971           │
│ funding_total_usd   ┆ 85.818583           │
│ first_funding_at    ┆ 83.970227           │
│ last_funding_at     ┆ 83.970227           │
│ funding_rounds      ┆ 83.868473           │
│ state_code          ┆ 74.102151           │
│ twitter_username    ┆ 58.997828           │
│ tag_list            ┆ 58.559778           │
│ lat         

In [3]:
# Keeping columns with relevant information and low number of nulls
cols_keep = [
    'name', 'country_code', 'region', 'city',
    'founded_at', 'created_at',
    'funding_total_usd', 'funding_rounds',
    'category_code', 'status'
]
df_companies_filtered = df_companies.select(cols_keep)

# Print the shape to confirm the number of columns
print(f"Original columns: {df_companies.width}")
print(f"Filtered columns: {df_companies_filtered.width}")

Original columns: 44
Filtered columns: 10


In [4]:
# Remove quotes AND strip whitespace

df_companies_filtered = df_companies_filtered.with_columns(
    pl.col(pl.Utf8)
      # 1. First transformation: Remove double quotes (")
      .str.replace_all('"', "", literal=True) # Added literal=True for Polars consistency
      # 2. Second transformation: Remove leading/trailing whitespace explicitly
      .str.strip_chars() 
      # 3. Keep the original column names
      .name.keep()
)

# Display the first few rows to verify both changes
print("--- DataFrame after removing quotes AND stripping whitespace ---")
print(df_companies_filtered.head())

--- DataFrame after removing quotes AND stripping whitespace ---
shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ name      ┆ country_c ┆ region    ┆ city      ┆ … ┆ funding_t ┆ funding_r ┆ category_ ┆ status   │
│ ---       ┆ ode       ┆ ---       ┆ ---       ┆   ┆ otal_usd  ┆ ounds     ┆ code      ┆ ---      │
│ str       ┆ ---       ┆ str       ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ str      │
│           ┆ str       ┆           ┆           ┆   ┆ f64       ┆ f64       ┆ str       ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Wetpaint  ┆ USA       ┆ Seattle   ┆ Seattle   ┆ … ┆ 3.975e7   ┆ 3.0       ┆ web       ┆ operatin │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ g        │
│ Flektor   ┆ USA       ┆ Los       ┆ Culver    ┆ … ┆ null      ┆ null      ┆ games_vid ┆ acquired │
│          

In [5]:
# Date Transformation Step: Replace original string columns with only the Year (Integer)

df_companies_filtered = df_companies_filtered.with_columns(
    # 1. Replace 'founded_at' with just the year (Integer)
    pl.col("founded_at")
      .str.to_datetime(format="%Y-%m-%d", strict=False) # Correct conversion: .str.to_datetime()
      .dt.year()                                       # Extract the year
      .alias("founded_at"),                            # Overwrite the original column

    # 2. Replace 'created_at' with just the year (Integer)
    pl.col("created_at")
      .str.to_datetime(format="%Y-%m-%d %H:%M:%S", strict=False)
      .dt.year()
      .alias("created_at"),
)

# Display the head to confirm columns now contain years (Integers)
print("--- DataFrame after Replacing Dates with Year (Integer) ---")
print(df_companies_filtered.head().select(['founded_at', 'created_at']))

# Display the dtypes to confirm they are now Integer type (i32)
print("\n--- Data Types Confirmation ---")
print(df_companies_filtered.select(['founded_at', 'created_at']).dtypes)

--- DataFrame after Replacing Dates with Year (Integer) ---
shape: (5, 2)
┌────────────┬────────────┐
│ founded_at ┆ created_at │
│ ---        ┆ ---        │
│ i32        ┆ i32        │
╞════════════╪════════════╡
│ 2005       ┆ 2007       │
│ null       ┆ 2007       │
│ null       ┆ 2007       │
│ 2008       ┆ 2008       │
│ 2008       ┆ 2008       │
└────────────┴────────────┘

--- Data Types Confirmation ---
[Int32, Int32]


In [6]:
# Location columns that might contain "unknown"
location_cols = ['region', 'city']

# Impute "unknown" string with Null for location columns
df_companies_filtered = df_companies_filtered.with_columns(
    [
        pl.when(pl.col(col) == "unknown") # Check if the value in the column equals the string "unknown"
          .then(None)                     # If TRUE, replace it with Polars Null (None)
          .otherwise(pl.col(col))         # Otherwise, keep the original value
          .alias(col)
        for col in location_cols
    ]
)

print("\n--- Null check after standardizing 'unknown' ---")
print(df_companies_filtered.select(location_cols).null_count())


--- Null check after standardizing 'unknown' ---
shape: (1, 2)
┌────────┬────────┐
│ region ┆ city   │
│ ---    ┆ ---    │
│ u32    ┆ u32    │
╞════════╪════════╡
│ 109866 ┆ 112663 │
└────────┴────────┘


In [7]:
print("--- Current State of 'funding_total_usd' ---")
print(df_companies_filtered.select(['funding_total_usd']).head(20))
print("\n--- Current Dtype ---")
print(df_companies_filtered.select('funding_total_usd').dtypes)

--- Current State of 'funding_total_usd' ---
shape: (20, 1)
┌───────────────────┐
│ funding_total_usd │
│ ---               │
│ f64               │
╞═══════════════════╡
│ 3.975e7           │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ 5e6               │
│ null              │
│ null              │
│ null              │
│ null              │
│ null              │
│ 6.80692e7         │
└───────────────────┘

--- Current Dtype ---
[Float64]


### Cleaning df_unicorn

In [8]:
# Set Polars display option to show all 44 rows (we set to 50 for safety)
# This uses the correct pl.Config method for your Polars version.
pl.Config.set_tbl_rows(50)

# --- Null Percentage Calculation for df_companies ---

# 2. Get the total number of rows
total_rows = df_unicorn.height

# 3. Select all columns, apply the null count logic, and calculate percentage
null_counts_df = df_unicorn.select(
    (pl.col("*").null_count() / total_rows) * 100
)

# 4. Extract column names and percentage values
column_names = null_counts_df.columns
percentage_values = null_counts_df.row(0) 

# 5. Create the final DataFrame
null_report_final = pl.DataFrame({
    "Column Name": column_names,
    "Null Percentage (%)": pl.Series(percentage_values)
})

# 6. Sort the report to see the highest percentage of nulls first
final_report_sorted = null_report_final.sort("Null Percentage (%)", descending=True)

# Print the final report (now showing all 44 rows)
print("--- Full Null Percentage Report (df_companies) ---")
print(final_report_sorted)

--- Full Null Percentage Report (df_companies) ---
shape: (13, 2)
┌───────────────────┬─────────────────────┐
│ Column Name       ┆ Null Percentage (%) │
│ ---               ┆ ---                 │
│ str               ┆ f64                 │
╞═══════════════════╪═════════════════════╡
│ Financial Stage   ┆ 95.274831           │
│ Portfolio Exits   ┆ 95.274831           │
│ Founded Year      ┆ 4.146577            │
│ Deal Terms        ┆ 2.796528            │
│ Total Raised      ┆ 2.314368            │
│ Select Inverstors ┆ 1.639344            │
│ Investors Count   ┆ 0.096432            │
│ Company           ┆ 0.0                 │
│ Valuation ($B)    ┆ 0.0                 │
│ Date Joined       ┆ 0.0                 │
│ Country           ┆ 0.0                 │
│ City              ┆ 0.0                 │
│ Industry          ┆ 0.0                 │
└───────────────────┴─────────────────────┘


In [9]:
# Keeping columns with relevant information and low number of nulls
cols_keep = [
    'Company', 'Valuation ($B)', 'Date Joined', 'Country', 'City', 
    'Industry', 'Select Inverstors', 'Founded Year', 'Total Raised', 
    'Investors Count', 'Deal Terms'
]
df_unicorn_filtered = df_unicorn.select(cols_keep)

# Print the shape to confirm the number of columns
print(f"Original columns: {df_unicorn.width}")
print(f"Filtered columns: {df_unicorn_filtered.width}")
df_unicorn_filtered

Original columns: 13
Filtered columns: 11


Company,Valuation ($B),Date Joined,Country,City,Industry,Select Inverstors,Founded Year,Total Raised,Investors Count,Deal Terms
str,str,str,str,str,str,str,i64,str,i64,i64
"""Bytedance""","""$140""","""4/7/2017""","""China""","""Beijing""","""Artificial intelligence""","""Sequoia Capital China, SIG Asi…",2012,"""$7.44B""",28,8
"""SpaceX""","""$100.3""","""12/1/2012""","""United States""","""Hawthorne""","""Other""","""Founders Fund, Draper Fisher J…",2002,"""$6.874B""",29,12
"""Stripe""","""$95""","""1/23/2014""","""United States""","""San Francisco""","""Fintech""","""Khosla Ventures, LowercaseCapi…",2010,"""$2.901B""",39,12
"""Klarna""","""$45.6""","""12/12/2011""","""Sweden""","""Stockholm""","""Fintech""","""Institutional Venture Partners…",2005,"""$3.472B""",56,13
"""Epic Games""","""$42""","""10/26/2018""","""United States""","""Cary""","""Other""","""Tencent Holdings, KKR, Smash V…",1991,"""$4.377B""",25,5
"""Canva""","""$40""","""1/8/2018""","""Australia""","""Surry Hills""","""Internet software & services""","""Sequoia Capital China, Blackbi…",2012,"""$571.26M""",26,8
"""Checkout.com""","""$40""","""5/2/2019""","""United Kingdom""","""London""","""Fintech""","""Tiger Global Management, Insig…",2012,"""$1.83B""",15,4
"""Instacart""","""$39""","""12/30/2014""","""United States""","""San Francisco""","""Supply chain, logistics, & del…","""Khosla Ventures, Kleiner Perki…",2012,"""$2.686B""",29,12
"""Databricks""","""$38""","""2/5/2019""","""United States""","""San Francisco""","""Data management & analytics""","""Andreessen Horowitz, New Enter…",2013,"""$3.497B""",29,8


In [10]:
# Remove quotes AND strip whitespace

df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col(pl.Utf8)
      # 1. First transformation: Remove double quotes (")
      .str.replace_all('"', "")
      # 2. Second transformation: Remove leading/trailing whitespace explicitly
      .str.strip_chars() 
      # 3. Keep the original column names
      .name.keep()
)

# Display the first few rows to verify both changes
print("--- DataFrame after removing quotes AND stripping whitespace ---")
print(df_unicorn_filtered.head())

--- DataFrame after removing quotes AND stripping whitespace ---
shape: (5, 11)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
│ Company    ┆ Valuation  ┆ Date       ┆ Country   ┆ … ┆ Founded   ┆ Total     ┆ Investors ┆ Deal  │
│ ---        ┆ ($B)       ┆ Joined     ┆ ---       ┆   ┆ Year      ┆ Raised    ┆ Count     ┆ Terms │
│ str        ┆ ---        ┆ ---        ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---   │
│            ┆ str        ┆ str        ┆           ┆   ┆ i64       ┆ str       ┆ i64       ┆ i64   │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
│ Bytedance  ┆ $140       ┆ 4/7/2017   ┆ China     ┆ … ┆ 2012      ┆ $7.44B    ┆ 28        ┆ 8     │
│ SpaceX     ┆ $100.3     ┆ 12/1/2012  ┆ United    ┆ … ┆ 2002      ┆ $6.874B   ┆ 29        ┆ 12    │
│            ┆            ┆            ┆ States    ┆   ┆           ┆           ┆           ┆       │
│ Stripe   

In [11]:
# Replace original string column 'Date Joined' with only the Year (Integer)

df_unicorn_filtered = df_unicorn_filtered.with_columns(
    # 1. Replace 'Date Joined' with just the year (Integer)
    pl.col("Date Joined")
      # CORRECTED FORMAT: Use %m/%d/%Y to match '4/7/2017'
      .str.to_datetime(format="%m/%d/%Y", strict=False) 
      .dt.year()                                       
      .alias("Date Joined")                            
)

# Display the head to confirm columns now contain years (Integers)
print("--- DataFrame after Replacing Dates with Year (Integer) ---")
print(df_unicorn_filtered.head().select(['Date Joined']))

--- DataFrame after Replacing Dates with Year (Integer) ---
shape: (5, 1)
┌─────────────┐
│ Date Joined │
│ ---         │
│ i32         │
╞═════════════╡
│ 2017        │
│ 2012        │
│ 2014        │
│ 2011        │
│ 2018        │
└─────────────┘


In [12]:
# Standardize column names (lowercase, replace spaces with _)

def standardize_col_name(name: str) -> str:
    """Converts column name to lowercase and replaces spaces with underscores."""
    # 1. Convert to lowercase
    name = name.lower()
    # 2. Replace spaces with _
    name = name.replace(' ', '_')
    # 3. Replace other characters that might cause issues (e.g., $, (, ), etc.)
    # This is optional but recommended for robustness
    for char in ['$', '(', ')', ',']:
        name = name.replace(char, '')
    
    return name

# Apply the standardization function to all column names
new_column_names = [standardize_col_name(col) for col in df_unicorn_filtered.columns]

# Create a mapping dictionary: {'Old Name': 'new_name'}
column_mapping = dict(zip(df_unicorn_filtered.columns, new_column_names))

# Rename the columns using .rename()
df_unicorn_filtered = df_unicorn_filtered.rename(column_mapping)


print("--- New Columns of df_unicorn_filtered ---")
print(df_unicorn_filtered.columns)
print("\n--- Head of DataFrame with Standardized Columns ---")
print(df_unicorn_filtered.head())

--- New Columns of df_unicorn_filtered ---
['company', 'valuation_b', 'date_joined', 'country', 'city', 'industry', 'select_inverstors', 'founded_year', 'total_raised', 'investors_count', 'deal_terms']

--- Head of DataFrame with Standardized Columns ---
shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ company   ┆ valuation ┆ date_join ┆ country   ┆ … ┆ founded_y ┆ total_rai ┆ investors ┆ deal_ter │
│ ---       ┆ _b        ┆ ed        ┆ ---       ┆   ┆ ear       ┆ sed       ┆ _count    ┆ ms       │
│ str       ┆ ---       ┆ ---       ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ str       ┆ i32       ┆           ┆   ┆ i64       ┆ str       ┆ i64       ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Bytedance ┆ $140      ┆ 2017      ┆ China     ┆ … ┆ 2012      ┆ $7.44B    ┆ 28        ┆ 8        │
│ SpaceX    ┆ $100.3   

In [13]:
# Function to clean symbols ($, ,) and strip whitespace
def clean_numeric_string(col_expr):
    """Removes $ and commas, and strips whitespace."""
    if isinstance(col_expr, str):
        col_expr = pl.col(col_expr)
    
    # We rely on previous global .str.strip_chars() but put it here for safety.
    return col_expr.str.replace_all('$', '', literal=True).str.replace_all(',', '', literal=True).str.strip_chars()

In [14]:
# Final Conversion Step: Reintroducing cast with strict=False to bypass complex expression error
# This block uses the clean_numeric_string function defined in the previous cell.

df_unicorn_filtered = df_unicorn_filtered.with_columns(
    
    # -------------------------------------------------------------
    # --- 1. Valuation ($B) ---
    # -------------------------------------------------------------
    clean_numeric_string("valuation_b")
      # FIX: Use strict=False to ensure Polars handles potential parsing issues gracefully (returns Null on fail).
      .cast(pl.Float64, strict=False) 
      .alias("valuation_usd_b"),

    # -------------------------------------------------------------
    # --- 2. Total Raised ($M/$B) - In Billions (B) ---
    # -------------------------------------------------------------
    pl.col("total_raised")
      .str.to_uppercase() 
      
      .pipe(lambda s: 
            # If 'B': Remove 'B', clean, and cast.
            pl.when(s.str.ends_with('B'))
              .then(clean_numeric_string(s).str.replace_all('B', '', literal=True).cast(pl.Float64, strict=False))
            
            # If 'M': Remove 'M', clean, cast, and divide by 1000.
            .when(s.str.ends_with('M'))
              .then(clean_numeric_string(s).str.replace_all('M', '', literal=True).cast(pl.Float64, strict=False) / 1000)
            
            # If neither M nor B: Clean the string and attempt final cast.
            .otherwise(
                clean_numeric_string(s).cast(pl.Float64, strict=False) 
            )
      )
      .alias("total_raised_usd_b")
)

# Display the results and dtypes to confirm
print("--- Conversão Final de Colunas Financeiras ---")
print(df_unicorn_filtered.head().select(['valuation_b', 'valuation_usd_b', 'total_raised', 'total_raised_usd_b']))
print("\n--- Dtypes Check for New Columns ---")
print(df_unicorn_filtered.select(['valuation_usd_b', 'total_raised_usd_b']).dtypes)

--- Conversão Final de Colunas Financeiras ---
shape: (5, 4)
┌─────────────┬─────────────────┬──────────────┬────────────────────┐
│ valuation_b ┆ valuation_usd_b ┆ total_raised ┆ total_raised_usd_b │
│ ---         ┆ ---             ┆ ---          ┆ ---                │
│ str         ┆ f64             ┆ str          ┆ f64                │
╞═════════════╪═════════════════╪══════════════╪════════════════════╡
│ $140        ┆ 140.0           ┆ $7.44B       ┆ 7.44               │
│ $100.3      ┆ 100.3           ┆ $6.874B      ┆ 6.874              │
│ $95         ┆ 95.0            ┆ $2.901B      ┆ 2.901              │
│ $45.6       ┆ 45.6            ┆ $3.472B      ┆ 3.472              │
│ $42         ┆ 42.0            ┆ $4.377B      ┆ 4.377              │
└─────────────┴─────────────────┴──────────────┴────────────────────┘

--- Dtypes Check for New Columns ---
[Float64, Float64]


In [15]:
print(df_unicorn_filtered.head()) 

shape: (5, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ company   ┆ valuation ┆ date_join ┆ country   ┆ … ┆ investors ┆ deal_term ┆ valuation ┆ total_ra │
│ ---       ┆ _b        ┆ ed        ┆ ---       ┆   ┆ _count    ┆ s         ┆ _usd_b    ┆ ised_usd │
│ str       ┆ ---       ┆ ---       ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ _b       │
│           ┆ str       ┆ i32       ┆           ┆   ┆ i64       ┆ i64       ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Bytedance ┆ $140      ┆ 2017      ┆ China     ┆ … ┆ 28        ┆ 8         ┆ 140.0     ┆ 7.44     │
│ SpaceX    ┆ $100.3    ┆ 2012      ┆ United    ┆ … ┆ 29        ┆ 12        ┆ 100.3     ┆ 6.874    │
│           ┆           ┆           ┆ States    ┆   ┆           ┆           

In [16]:
# Convert 'investors_count' to Int64, handling potential non-numeric strings with strict=False.
df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col("investors_count")
      .cast(pl.Int64, strict=False)
      .alias("investors_count")
)

In [17]:
# Unique values of 'select_inverstors'
unique_investors_count = df_unicorn_filtered.select(
    pl.col("select_inverstors").n_unique()
).item() 
print(f"The column 'select_inverstors' has {unique_investors_count} unique values.")

The column 'select_inverstors' has 1006 unique values.


In [18]:
# Inspection Step: Check types and unique values for remaining columns

print("--- Dtypes Check (Focusing on Remaining Columns) ---")
# We include 'investors_count' here to confirm the Int64 conversion took place
print(df_unicorn_filtered.select(['deal_terms', 'select_inverstors', 'date_joined', 'founded_year', 'investors_count']).dtypes)

print("\n--- Unique Values for 'deal_terms' (Top 10) ---")
# Check the content of deal_terms
try:
    print(df_unicorn_filtered.select('deal_terms').value_counts().head(10))
except Exception:
    print("Could not display value counts for deal_terms. Moving on.")


print("\n--- Unique Values for 'portfolio_exits' (Top 10) ---")
# Check if 'portfolio_exits' exists and its values
try:
    print(df_unicorn_filtered.select('portfolio_exits').value_counts().head(10))
except Exception as e:
    print(f"Could not check 'portfolio_exits'. It might not exist or is named differently. Error: {e}")

--- Dtypes Check (Focusing on Remaining Columns) ---
[Int64, String, Int32, Int64, Int64]

--- Unique Values for 'deal_terms' (Top 10) ---
Could not display value counts for deal_terms. Moving on.

--- Unique Values for 'portfolio_exits' (Top 10) ---
Could not check 'portfolio_exits'. It might not exist or is named differently. Error: unable to find column "portfolio_exits"; valid columns: ["company", "valuation_b", "date_joined", "country", "city", "industry", "select_inverstors", "founded_year", "total_raised", "investors_count", "deal_terms", "valuation_usd_b", "total_raised_usd_b"]


In [19]:
# Clean and Convert 'deal_terms' to Integer (Int64) using sequential literal replacements.
df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col("deal_terms")
      .cast(pl.String)  # Convert to String first for cleaning
      # FIX: Use multiple literal replace_all calls instead of regex=True
      .str.replace_all("-", "", literal=True)
      .str.replace_all("/", "", literal=True)
      .str.replace_all("NA", "", literal=True) 
      .str.strip_chars() # Remove any remaining whitespace
      .cast(pl.Int64, strict=False) # Convert to Int64, replacing failed conversions with Null
      .alias("deal_terms")
)
print("--- 'deal_terms' has been cleaned and cast to Int64 (Nulls introduced for bad values) ---")

--- 'deal_terms' has been cleaned and cast to Int64 (Nulls introduced for bad values) ---


In [20]:
# Define a final column order and remove redundant/missing columns.
final_column_order_unicorn = [
    "company",
    "country",
    "city",
    "industry",
    "date_joined",
    "founded_year",
    "valuation_usd_b",          # Cleaned Float64 (Billion USD)
    "total_raised_usd_b",       # Cleaned Float64 (Billion USD)
    "investors_count",          # Cleaned Int64
    "deal_terms",               # Cleaned Int64 (or Null)
    "select_inverstors",        # Textual (to be kept as is)
]

# Apply the selection and reordering.
df_unicorn_filtered = df_unicorn_filtered.select(final_column_order_unicorn)

# Display the final, cleaned DataFrame structure.
print("\n--- Final Cleaned and Reordered DataFrame (df_unicorn_filtered) ---")
print(df_unicorn_filtered.head())
print("\n--- Final Dtypes for df_unicorn_filtered ---")
print(df_unicorn_filtered.dtypes)


--- Final Cleaned and Reordered DataFrame (df_unicorn_filtered) ---
shape: (5, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ company   ┆ country   ┆ city      ┆ industry  ┆ … ┆ total_rai ┆ investors ┆ deal_term ┆ select_i │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ sed_usd_b ┆ _count    ┆ s         ┆ nverstor │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ s        │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆ i64       ┆ i64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Bytedance ┆ China     ┆ Beijing   ┆ Artificia ┆ … ┆ 7.44      ┆ 28        ┆ 8         ┆ Sequoia  │
│           ┆           ┆           ┆ l intelli ┆   ┆           ┆           ┆           ┆ Capital  │
│      

### Merging dfs

In [21]:
# Final Step: Rename and Drop for Merge Preparation

# 1. Renomear as colunas para padronização
df_companies_filtered = df_companies_filtered.rename({
    "name": "company",
    "country_code": "country",
    "founded_at": "founded_year",
    "category_code": "industry",
})

# 2. Remover colunas desnecessárias
df_companies_filtered = df_companies_filtered.drop(['created_at', 'region'])

# 3. Listar as colunas novamente para confirmar que a ação foi bem-sucedida
print("--- SUCESSO: Colunas de df_companies_filtered após Renomeação ---")
print(df_companies_filtered.columns)

print("\n--- df_unicorn_filtered (para comparação) ---")
print(df_unicorn_filtered.columns)

--- SUCESSO: Colunas de df_companies_filtered após Renomeação ---
['company', 'country', 'city', 'founded_year', 'funding_total_usd', 'funding_rounds', 'industry', 'status']

--- df_unicorn_filtered (para comparação) ---
['company', 'country', 'city', 'industry', 'date_joined', 'founded_year', 'valuation_usd_b', 'total_raised_usd_b', 'investors_count', 'deal_terms', 'select_inverstors']


In [22]:
# 1. Extract ALL unique countries from df_unicorn_filtered (Long Names)
unicorn_countries_list = df_unicorn_filtered.select('country').unique().to_series().to_list()

# 2. Extract ALL unique countries/codes from df_companies_filtered (Codes/Mixed)
companies_countries_list = df_companies_filtered.select('country').unique().to_series().to_list()

print("--- Full List of Unique Countries in df_unicorn_filtered (Long Names) ---")
print(unicorn_countries_list)
print(f"Total Unique Countries in Unicorn DF: {len(unicorn_countries_list)}\n")

print("--- Full List of Unique Countries in df_companies_filtered (Codes/Mixed) ---")
print(companies_countries_list)
print(f"Total Unique Countries in Companies DF: {len(companies_countries_list)}")

--- Full List of Unique Countries in df_unicorn_filtered (Long Names) ---
['Indonesia', 'Chile', 'Nigeria', 'Sweden', 'Croatia', 'Colombia', 'Czech Republic', 'Denmark', 'Brazil', 'Switzerland', 'Malaysia', 'United Kingdom', 'Norway', 'Luxembourg', 'Italy', 'Bahamas', 'South Africa', 'Turkey', 'Ireland', 'Vietnam', 'Australia', 'Japan', 'Bermuda', 'Estonia', 'United States', 'United Arab Emirates', 'Senegal', 'Hong Kong', 'Lithuania', 'India', 'Belgium', 'Philippines', 'Argentina', 'Israel', 'Singapore', 'Thailand', 'Canada', 'South Korea', 'Spain', 'Germany', 'France', 'Austria', 'Netherlands', 'China', 'Finland', 'Mexico']
Total Unique Countries in Unicorn DF: 46

--- Full List of Unique Countries in df_companies_filtered (Codes/Mixed) ---
['BEN', 'COL', 'HMI', 'AFG', 'BRN', 'SYC', 'SEN', 'CRI', 'FST', 'GRD', 'ISR', 'BRB', 'BGR', 'PHL', 'MMR', 'MDG', 'VIR', 'ISL', 'LBN', 'MKD', 'IOT', 'NIC', 'CSS', 'BMU', 'HUN', 'URY', 'TJK', 'LVA', 'CHL', 'ARG', 'NGA', 'DEU', 'ETH', 'TZA', 'BOL', 'U

In [23]:
# The COMPLETE mapping dictionary (including all common ISO codes)
country_mapping_complete = {
    "USA": "United States", 
    "CHN": "China",
    "GBR": "United Kingdom",
    "DEU": "Germany",
    "ISR": "Israel",
    "SEN": "Senegal",
    "CHL": "Chile",
    "HRV": "Croatia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "BRA": "Brazil",
    "PHL": "Philippines",
    "MEX": "Mexico",
    "MYS": "Malaysia",
    "ARG": "Argentina",
    "NOR": "Norway",
    "ZAF": "South Africa",
    "SWE": "Sweden",
    "ITA": "Italy",
    "FIN": "Finland",
    "CHE": "Switzerland",
    "IND": "India",
    "SGP": "Singapore",
    "BHS": "Bahamas",
    "COL": "Colombia",
    "LTU": "Lithuania",
    "KOR": "South Korea",
    "EST": "Estonia",
    "DNK": "Denmark",
    "IRL": "Ireland",
    "VNM": "Vietnam",
    "CAN": "Canada",
    "CZE": "Czech Republic",
    "ARE": "United Arab Emirates",
    "IDN": "Indonesia",
    "LUX": "Luxembourg",
    "THA": "Thailand",
    "ESP": "Spain",
    "JPN": "Japan",
    "AUS": "Australia",
    "NLD": "Netherlands",
    "HKG": "Hong Kong",
    "BMU": "Bermuda",
    "NGA": "Nigeria",
    "FRA": "France",
    "TUR": "Turkey"
}

# 1. Apply the country standardization
df_companies_filtered = df_companies_filtered.with_columns(
    pl.col("country")
      .replace(country_mapping_complete, default=pl.col("country")) # Replace codes; if not found, keeps original value
      .alias("country")
)

# 2. Standardize the case of the merge key in BOTH DataFrames
df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col("company").str.to_titlecase().alias("company")
)
df_companies_filtered = df_companies_filtered.with_columns(
    pl.col("company").str.to_titlecase().alias("company")
)

/var/folders/f1/77pps9rs2l9ffb8_p5tj2yrr0000gp/T/ipykernel_38556/799306067.py:53: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  pl.col("country")


In [24]:
# 1. Extract the UNIQUE company names as Python sets
# Set A: Unicorns
set_a_name = df_unicorn_filtered.select('company').unique().to_series().to_list()
set_a = set(set_a_name)

# Set B: Companies
set_b_name = df_companies_filtered.select('company').unique().to_series().to_list()
set_b = set(set_b_name)

# 2. Determine the smaller set
if len(set_a) <= len(set_b):
    smaller_set = set_a
    larger_set = set_b
    set_a_label = "Unicorns"
    set_b_label = "Companies"
else:
    smaller_set = set_b
    larger_set = set_a
    set_a_label = "Companies"
    set_b_label = "Unicorns"

# 3. Calculate Intersection (Common Elements)
common_elements = smaller_set.intersection(larger_set)
count_common = len(common_elements)

# 4. Calculate Difference (Elements in Smaller Set NOT in Larger Set)
# This matches your example: "não em comum da lista que tem menos elementos"
unique_to_smaller_set = smaller_set.difference(larger_set)
count_unique_to_smaller = len(unique_to_smaller_set)

# 5. Calculate Percentage
# Percentage of the smaller set that is NOT common (the difference)
if len(smaller_set) > 0:
    percentage_unique = (count_unique_to_smaller / len(smaller_set)) * 100
else:
    percentage_unique = 0.0

# --- Display Results ---
print("--- Set Analysis on Company Names ---")
print(f"Total Unique Companies in Unicorn DF (Set A): {len(set_a)}")
print(f"Total Unique Companies in Companies DF (Set B): {len(set_b)}")
print("-" * 40)

print(f"Count of Common Elements (Intersection): {count_common}")
print(f"Count of Elements Unique to the Smaller Set: {count_unique_to_smaller}")
print("-" * 40)

print(f"The Smaller Set ({set_a_label} or {set_b_label}) has {len(smaller_set)} elements.")
print(f"Percentage of the Smaller Set NOT in Common: {percentage_unique:.2f}%")

# Optional: Display the actual unique elements from the smaller set
# print(f"\nElements Unique to the Smaller Set ({set_a_label if len(set_a) <= len(set_b) else set_b_label}):")
# print(list(unique_to_smaller_set)[:20]) # Displaying first 20 for brevity

--- Set Analysis on Company Names ---
Total Unique Companies in Unicorn DF (Set A): 1034
Total Unique Companies in Companies DF (Set B): 196325
----------------------------------------
Count of Common Elements (Intersection): 247
Count of Elements Unique to the Smaller Set: 787
----------------------------------------
The Smaller Set (Unicorns or Companies) has 1034 elements.
Percentage of the Smaller Set NOT in Common: 76.11%


In [25]:
# Apply basic cleaning: lowercase and strip leading/trailing whitespace
df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col("industry").str.to_lowercase().str.strip_chars().alias("industry")
)

df_companies_filtered = df_companies_filtered.with_columns(
    pl.col("industry").str.to_lowercase().str.strip_chars().alias("industry")
)

print("--- Basic Industry Cleaning Applied ---")
print(df_unicorn_filtered.select('industry').sample(5))
print(df_companies_filtered.select('industry').sample(5))

--- Basic Industry Cleaning Applied ---
shape: (5, 1)
┌─────────────────────────────────┐
│ industry                        │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ e-commerce & direct-to-consume… │
│ fintech                         │
│ artificial intelligence         │
│ consumer & retail               │
│ fintech                         │
└─────────────────────────────────┘
shape: (5, 1)
┌─────────────┐
│ industry    │
│ ---         │
│ str         │
╞═════════════╡
│ games_video │
│ null        │
│ education   │
│ null        │
│ other       │
└─────────────┘


In [26]:
# Ensure the configuration from the previous step is still active if possible, 
# but we will explicitly select the column for cleaner output.

print("--- Unique Industries in df_unicorn_filtered (After Basic Clean) ---")
# Select only the industry column, get unique values, sort, and convert to a list for clean display
unicorn_industries = df_unicorn_filtered.select('industry').unique().sort('industry').to_series().to_list()
for industry in unicorn_industries:
    print(industry)
print(f"\nTotal Unique Industries in Unicorn DF: {len(unicorn_industries)}\n")


print("--- Unique Industries in df_companies_filtered (After Basic Clean) ---")
# Display industries from the companies DF
companies_industries = df_companies_filtered.select('industry').unique().sort('industry').to_series().to_list()
for industry in companies_industries:
    print(industry)
print(f"\nTotal Unique Industries in Companies DF: {len(companies_industries)}")

--- Unique Industries in df_unicorn_filtered (After Basic Clean) ---
500 global, rakuten ventures, golden gate ventures
andreessen horowitz, dst global, idg capital
artificial intelligence
auto & transportation
b capital group, monk's hill ventures, dynamic parcel distribution
consumer & retail
cybersecurity
data management & analytics
dragonfly captial, qiming venture partners, dst global
e-commerce & direct-to-consumer
edtech
fintech
finttech
hardware
health
hopu investment management, boyu capital, dc thomson ventures
internet software & services
jungle ventures, accel, venture highway
kuang-chi
mobile & telecommunications
mundi ventures, doqling capital partners, activant capital
other
sequoia capital china, ing, alibaba entrepreneurs fund
sequoia capital china, shunwei capital partners, qualgro
sequoia capital, thoma bravo, softbank
singtel innov8, alpha jwc ventures, golden gate ventures
supply chain, logistics, & delivery
temasek, guggenheim investments, qatar investment authori

In [29]:
import polars as pl

# --- Industry Cleaning and Filtering for df_unicorn_filtered ---

# 1. Definitive List of Valid Industries from the Unicorn DF (Target Categories)
valid_unicorn_industries = [
    'artificial intelligence',
    'auto & transportation',
    'consumer & retail',
    'cybersecurity',
    'data management & analytics',
    'e-commerce & direct-to-consumer',
    'edtech',
    'fintech',
    'finttech',  # Kept here temporarily to match the filter list
    'hardware',
    'health',
    'internet software & services',
    'mobile & telecommunications',
    'supply chain, logistics, & delivery',
    'travel',
    'other'
]

# 2. Typo Correction Mapping (Fixing 'finttech' -> 'fintech')
industry_correction_map = {
    'finttech': 'fintech'
}

# 3. Filter df_unicorn_filtered (Removes rows where industry is an investor name)
df_unicorn_filtered = df_unicorn_filtered.filter(
    pl.col("industry").is_in(valid_unicorn_industries)
)

# 4. Apply the typo correction to the remaining industry column
df_unicorn_filtered = df_unicorn_filtered.with_columns(
    pl.col("industry").replace(industry_correction_map).alias("industry")
)

print(f"Number of rows remaining in df_unicorn_filtered after industry filtering: {df_unicorn_filtered.shape[0]}")

# --- Verification Step ---
# Print the final, unique list of industries from the cleaned DataFrame
print("\n--- Final Complete Unique Industries in df_unicorn_filtered ---")
final_unicorn_industries_list = df_unicorn_filtered.select('industry').unique().sort('industry').to_series().to_list()
for industry in final_unicorn_industries_list:
    print(industry)

print(f"\nTotal Unique Industries (Cleaned): {len(final_unicorn_industries_list)}")

Number of rows remaining in df_unicorn_filtered after industry filtering: 172

--- Final Complete Unique Industries in df_unicorn_filtered ---
hardware
health
other
travel

Total Unique Industries (Cleaned): 4


In [30]:
import polars as pl

# 1. MAPPING: From Unicorn Industry Names to Companies DF Industry Names
unicorn_to_companies_map = {
    'artificial intelligence': 'software',
    'auto & transportation': 'transportation',
    'consumer & retail': 'ecommerce',
    'cybersecurity': 'security',
    'data management & analytics': 'analytics',
    'e-commerce & direct-to-consumer': 'ecommerce',
    'edtech': 'education',
    'fintech': 'finance',
    'finttech': 'finance', # Fixes typo
    'hardware': 'hardware',
    'health': 'health',
    'internet software & services': 'software',
    'mobile & telecommunications': 'mobile',
    'other': 'other',
    'supply chain, logistics, & delivery': 'transportation',
    'travel': 'travel'
}

# 2. INVESTOR REGEX: (Used to identify and map non-industry entries to 'other')
investor_keywords_regex = (
    "500 global|rakuten ventures|andreessen horowitz|dst global|idg capital|"
    "dragonfly captial|qiming venture partners|hopu investment management|boyu capital|"
    "dc thomson ventures|jungle ventures|accel|venture highway|kuang-chi|mundi ventures|"
    "doqling capital partners|activant capital|sequoia capital china|ing|alibaba entrepreneurs fund|"
    "shunwei capital partners|qualgro|sequoia capital|thoma bravo|softbank|singtel innov8|"
    "alpha jwc ventures|temasek|guggenheim investments|qatar investment authority|"
    "tiger global management|tiger brokers|dcm ventures|vertex ventures se asia|global founders capital|"
    "visa ventures|vision plus capital|gsr ventures|zhenfund"
)

# --- Applying Transformations to df_unicorn_filtered (FIXED AGAIN with .replace()) ---

# We'll use a two-step process to ensure the logic is clean and correct:
# Step 1: Handle all investor rows and map them to 'other'.
# Step 2: On the rows that are NOT investors, apply the industry renaming map.

df_unicorn_temp = df_unicorn_filtered.with_columns(
    pl.when(pl.col("industry").str.contains(investor_keywords_regex))
      .then(pl.lit("other")) # Map investor rows to 'other'
      .otherwise(pl.col("industry")) # Keep the original industry name for non-investor rows
      .alias("industry_cleaned")
)

df_unicorn_filtered = df_unicorn_temp.with_columns(
    # Use .replace() to apply the dictionary map to the cleaned industry column.
    # The 'default' parameter ensures that if a value is NOT in the map (e.g., if it's 'other' from the previous step), 
    # it keeps its value.
    pl.col("industry_cleaned").replace(unicorn_to_companies_map, default=pl.col("industry_cleaned"))
    .alias("industry")
).drop(["industry_cleaned"])


# --- Verification ---
print("--- Verification: Industries in df_unicorn_filtered after Mapping (FINAL FIX) ---")
print(df_unicorn_filtered.select('industry').unique().sort('industry'))

--- Verification: Industries in df_unicorn_filtered after Mapping (FINAL FIX) ---
shape: (4, 1)
┌──────────┐
│ industry │
│ ---      │
│ str      │
╞══════════╡
│ hardware │
│ health   │
│ other    │
│ travel   │
└──────────┘


/var/folders/f1/77pps9rs2l9ffb8_p5tj2yrr0000gp/T/ipykernel_38556/2310645179.py:52: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  pl.col("industry_cleaned").replace(unicorn_to_companies_map, default=pl.col("industry_cleaned"))


In [31]:
df_unicorn_filtered

company,country,city,industry,date_joined,founded_year,valuation_usd_b,total_raised_usd_b,investors_count,deal_terms,select_inverstors
str,str,str,str,i32,i64,f64,f64,i64,i64,str
"""Spacex""","""United States""","""Hawthorne""","""other""",2012,2002,100.3,6.874,29,12,"""Founders Fund, Draper Fisher J…"
"""Epic Games""","""United States""","""Cary""","""other""",2018,1991,42.0,4.377,25,5,"""Tencent Holdings, KKR, Smash V…"
"""Dji Innovations""","""China""","""Shenzhen""","""hardware""",2015,2006,15.0,1.135,7,3,"""Accel Partners, Sequoia Capita…"
"""Devoted Health""","""United States""","""Waltham""","""health""",2018,2017,12.6,1.969,16,4,"""Andreessen Horowitz, F-Prime C…"
"""Bitmain Technologies""","""China""","""Beijing""","""hardware""",2018,2015,12.0,1.49,28,7,"""Coatue Management, Sequoia Cap…"
"""Biosplice Therapeutics""","""United States""","""San Diego""","""health""",2018,2008,12.0,0.5615,10,1,"""Vickers Venture Partners, IKEA…"
"""Global Switch""","""United Kingdom""","""London""","""hardware""",2016,1998,11.1,6.255,6,2,"""Aviation Industry Corporation …"
"""Thrasio""","""United States""","""Walpole""","""other""",2020,2018,10.0,3.396,22,5,"""Upper90, RiverPark Ventures, A…"
"""Heytea""","""China""","""Shenzhen""","""other""",2019,2012,9.28,0.57923,13,3,"""Sequoia Capital China, Tencent…"
